In [1]:
import cantera as ct
import re
import os 
from rmgpy.molecule import Molecule

In [20]:
gas = ct.Solution('./SI_from_paper/JPCA_PFCAThermo_12_16_2023.yaml')

In [25]:
index=-1 #start at 0

final_string = ''

for spc in gas.species():
    
    #increment the index 
    index+=1
    
    #get the label
    label = spc.name
    
    #get the adjacency list
    try: 
        SMILES = species_structure_dict[spc.name]
        adjacency_list = Molecule(smiles=SMILES).to_adjacency_list()
    except KeyError: 
        value = species_structure_dict_leftover[spc.name]
        if len(value)==1: 
            SMILES = value[0]
            adjacency_list = Molecule(smiles=SMILES).to_adjacency_list()
        if len(value)==2: 
            SMILES = value[0]
            adjacency_list = value[1]
    
    
    #now for the thermo
    max_temp = spc.thermo.max_temp
    min_temp = spc.thermo.min_temp
    middle_temp = spc.thermo.coeffs[0]
    first_T_region = list(gas.species(1).thermo.coeffs[-7:])
    second_T_region = list(spc.thermo.coeffs[1:8])
    cp0 = gas.species(1).thermo.cp(0)/1000 #cantera has it in J/kmol/K
    
    
    #source
    try: 
        note = spc.thermo.input_data['note']
        string = f'''

entry(
    index = {index},
    label = "{label}",
    molecule = 
"""
{adjacency_list}
""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs={first_T_region}, Tmin=({min_temp},'K'), Tmax=({middle_temp},'K')),
            NASAPolynomial(coeffs={second_T_region}, Tmin=({middle_temp},'K'), Tmax=({max_temp},'K')),
        ],
        Tmin = ({min_temp},'K'),
        Tmax = ({max_temp},'K'),
        Cp0 = ({cp0},'J/(mol*K)'),
    ),
    reference = '{note}',
    referenceType = "Theory",
)
'''
        final_string+=string
    except KeyError:
        string = f'''

entry(
    index = {index},
    label = "{label}",
    molecule = 
"""
{adjacency_list}
""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs={first_T_region}, Tmin=({min_temp},'K'), Tmax=({middle_temp},'K')),
            NASAPolynomial(coeffs={second_T_region}, Tmin=({middle_temp},'K'), Tmax=({max_temp},'K')),
        ],
        Tmin = ({min_temp},'K'),
        Tmax = ({max_temp},'K'),
        Cp0 = ({cp0},'J/(mol*K)'),
    ),
    referenceType = "Theory",
)
'''

# Get the structures! 

## Get the species from the first file! 

In [2]:
with open('./SI_from_paper/NCSU_PFAS_EXTRAPOLATED_THERMO_FEB_2024.dat', 'r') as f:
    lines = f.readlines()

In [5]:
species_lines = lines[26:182]

In [6]:
species_lines

['! Perfluorinated Carboxylic Acids\n',
 'PF9acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF10acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF11acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF12acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF13acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF14acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF15acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 'PF16acid ! O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F\n',
 '\n',
 '! Perfluorinated "Carboxylate Radicals"\n',
 'PF9carboxylr ! [O]C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)

In [7]:
species_lines = [line for line in species_lines if line[0]!='!' and line!='\n']

In [10]:
species_structure_dict = {}

for line in species_lines:
    matches = re.search('^(\S+) \! (\S+)', line)
    species_structure_dict[matches.group(1)]=matches.group(2)

## Get the species from the second file! 

In [12]:
with open('./SI_from_paper/NCSU_PFAS_THERMO_FEB_2024.dat', 'r') as f:
    lines = f.readlines()

In [13]:
species_lines = lines[38:302]

In [14]:
species_lines

['! Atomic Species\n',
 'H ! [H]\n',
 'O ! [O]\n',
 'F ! [F]\n',
 '\n',
 '! Diatomic Species\n',
 'H2 ! [H][H]\n',
 'O2 ! [O][O]\n',
 'F2 ! FF\n',
 '\n',
 '! H-O-F Species\n',
 'OHr ! [OH]\n',
 'H2O ! O\n',
 'HO2r ! [O]O\n',
 'H2O2 ! OO\n',
 'HF ! F\n',
 '\n',
 '! H-O-S Species\n',
 'SO3Hr ! O[S](=O)=O\n',
 'SO ! O=S\n',
 'SO2 ! O=S=O\n',
 'SO3 ! O=S(=O)=O\n',
 '\n',
 '! H-C1-O Species\n',
 'CO ! [C-]#[O+]\n',
 'CO2 ! O=C=O\n',
 'HOCOr ! O=[C]O\n',
 'HCOOr ! [O]C=O\n',
 'CH4 ! C\n',
 '\n',
 '! H-C1-O-F Species\n',
 'CF2(s) ! F[C]F, singlet\n',
 'CF2(t) ! F[C]F, triplet\n',
 'CF3r ! F[C](F)F\n',
 'CHF2r ! F[CH]F\n',
 'CH2F2 ! FCF\n',
 'CHF3 ! FC(F)F\n',
 'CF4 ! FC(F)(F)F\n',
 'CFOr ! O=[C]F\n',
 'CF2O ! O=C(F)F\n',
 'CF2OHr ! O[C](F)F\n',
 '\n',
 '! Misc. H-C2-O-F Species\n',
 'HOCF2CO ! O=[C]C(O)(F)F\n',
 'CHF2CO2 ! [O]C(=O)C(F)F\n',
 '\n',
 '! Perfluorinated Carboxylic Acids (PFCAs)\n',
 'FCOOH ! O=C(O)F\n',
 'PF2acid ! O=C(O)C(F)(F)F\n',
 'PF3acid ! O=C(O)C(F)(F)C(F)(F)F\n',
 'PF4aci

In [17]:
species_lines = [line for line in species_lines if line[0]!='!' and line!='\n']

In [18]:
for line in species_lines:
    matches = re.search('^(\S+) \! (\S+)', line)
    try: 
        species_structure_dict[matches.group(1)]=matches.group(2)
    except AttributeError: 
        print(line)

Final Dictionary! 

In [19]:
species_structure_dict

{'PF9acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF10acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF11acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF12acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF13acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF14acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF15acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF16acid': 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF9carboxylr': '[O]C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
 'PF10carboxylr': '[O]C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F

In [24]:
#these are all left out
species_structure_dict_leftover = {}

species_structure_dict_leftover['CF2s']=['F[C]F', 'multiplicity 1\n1 F u0 p3 c0 {2,S}\n2 C u0 p1 c0 {1,S} {3,S}\n3 F u0 p3 c0 {2,S}\n'] #singlet
species_structure_dict_leftover['CF2t']=['F[C]F', 'multiplicity 3\n1 F u0 p3 c0 {2,S}\n2 C u2 p0 c0 {1,S} {3,S}\n3 F u0 p3 c0 {2,S}\n' ]#triplet make sure
species_structure_dict_leftover['CHF2']=['[CH](F)F', 'multiplicity 2\n1 C u1 p0 c0 {2,S} {3,S} {4,S}\n2 F u0 p3 c0 {1,S}\n3 F u0 p3 c0 {1,S}\n4 H u0 p0 c0 {1,S}\n']
species_structure_dict_leftover['PF2enylr']=['F[C]=C(F)F', 'multiplicity 2\n1 F u0 p3 c0 {2,S}\n2 C u1 p0 c0 {1,S} {3,D}\n3 C u0 p0 c0 {2,D} {4,S} {5,S}\n4 F u0 p3 c0 {3,S}\n5 F u0 p3 c0 {3,S}\n']
species_structure_dict_leftover['PF2s']=['F[C]C(F)(F)F', 'multiplicity 1\n1 F u0 p3 c0 {2,S}\n2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}\n3 F u0 p3 c0 {2,S}\n4 F u0 p3 c0 {2,S}\n5 C u0 p1 c0 {2,S} {6,S}\n6 F u0 p3 c0 {5,S}\n']
species_structure_dict_leftover['CF3COr']=['O=[C]C(F)(F)F', 'multiplicity 2\n1 O u0 p2 c0 {2,D}\n2 C u1 p0 c0 {1,D} {3,S}\n3 C u0 p0 c0 {2,S} {4,S} {5,S} {6,S}\n4 F u0 p3 c0 {3,S}\n5 F u0 p3 c0 {3,S}\n6 F u0 p3 c0 {3,S}\n']
species_structure_dict_leftover['rCF2CFO']= ['[O]C(F)=C(F)F', 'multiplicity 2\n1 C u0 p0 c0 {2,D} {3,S} {4,S}\n2 C u0 p0 c0 {1,D} {5,S} {6,S}\n3 F u0 p3 c0 {1,S}\n4 F u0 p3 c0 {1,S}\n5 F u0 p3 c0 {2,S}\n6 O u1 p2 c0 {2,S}\n']
species_structure_dict_leftover['rCF2COOH']= ['	O=C(O)[C](F)F', 'multiplicity 2\n1 C u1 p0 c0 {2,S} {3,S} {4,S}\n2 C u0 p0 c0 {1,S} {5,S} {6,D}\n3 F u0 p3 c0 {1,S}\n4 F u0 p3 c0 {1,S}\n5 O u0 p2 c0 {2,S} {7,S}\n6 O u0 p2 c0 {2,D}\n7 H u0 p0 c0 {5,S}\n']
species_structure_dict_leftover['PF3-1ene']= ['FC(F)=C(F)C(F)(F)F', '1 F u0 p3 c0 {2,S}\n2 C u0 p0 c0 {1,S} {3,S} {4,D}\n3 F u0 p3 c0 {2,S}\n4 C u0 p0 c0 {2,D} {5,S} {6,S}\n5 F u0 p3 c0 {4,S}\n6 C u0 p0 c0 {4,S} {7,S} {8,S} {9,S}\n7 F u0 p3 c0 {6,S}\n8 F u0 p3 c0 {6,S}\n9 F u0 p3 c0 {6,S}\n']                                           
species_structure_dict_leftover['PF3s']= ['F[C]C(F)(F)C(F)(F)F', 'multiplicity 1\n1 F u0 p3 c0 {2,S}\n2 C u0 p1 c0 {1,S} {3,S}\n3 C u0 p0 c0 {2,S} {4,S} {5,S} {6,S}\n4 F u0 p3 c0 {3,S}\n5 F u0 p3 c0 {3,S}\n6 C u0 p0 c0 {3,S} {7,S} {8,S} {9,S}\n7 F u0 p3 c0 {6,S}\n8 F u0 p3 c0 {6,S}\n9 F u0 p3 c0 {6,S}\n']
species_structure_dict_leftover['rPF3acylF']= ['O=C(F)C(F)(F)[C](F)F', 'multiplicity 2\n1 O u0 p2 c0 {2,D}\n2 C u0 p0 c0 {1,D} {3,S} {4,S}\n3 F u0 p3 c0 {2,S}\n4 C u0 p0 c0 {2,S} {5,S} {6,S} {7,S}\n5 F u0 p3 c0 {4,S}\n6 F u0 p3 c0 {4,S}\n7 C u1 p0 c0 {4,S} {8,S} {9,S}\n8 F u0 p3 c0 {7,S}\n9 F u0 p3 c0 {7,S}\n']
species_structure_dict_leftover['rPF3acid']= ['	O=C(O)C(F)(F)[C](F)F', 'multiplicity 2\n1  O u0 p2 c0 {2,D}\n2  C u0 p0 c0 {1,D} {3,S} {4,S}\n3  O u0 p2 c0 {2,S} {10,S}\n4  C u0 p0 c0 {2,S} {5,S} {6,S} {7,S}\n5  F u0 p3 c0 {4,S}\n6  F u0 p3 c0 {4,S}\n7  C u1 p0 c0 {4,S} {8,S} {9,S}\n8  F u0 p3 c0 {7,S}\n9  F u0 p3 c0 {7,S}\n10 H u0 p0 c0 {3,S}\n']
species_structure_dict_leftover['rPF4-1ene']= ['F[C](F)C(F)(F)C(F)=C(F)F','multiplicity 2\n1  F u0 p3 c0 {2,S}\n2  C u0 p0 c0 {1,S} {3,S} {4,D}\n3  F u0 p3 c0 {2,S}\n4  C u0 p0 c0 {2,D} {5,S} {6,S}\n5  F u0 p3 c0 {4,S}\n6  C u0 p0 c0 {4,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u1 p0 c0 {6,S} {10,S} {11,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n']
species_structure_dict_leftover['PF4-1ene']= ['FC(F)=C(F)C(F)(F)C(F)(F)F', '1  F u0 p3 c0 {2,S}\n2  C u0 p0 c0 {1,S} {3,S} {4,D}\n3  F u0 p3 c0 {2,S}\n4  C u0 p0 c0 {2,D} {5,S} {6,S}\n5  F u0 p3 c0 {4,S}\n6  C u0 p0 c0 {4,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 F u0 p3 c0 {9,S}\n']
species_structure_dict_leftover['PF4s']= ['F[C]C(F)(F)C(F)(F)C(F)(F)F', 'multiplicity 1\n1  F u0 p3 c0 {2,S}\n2  C u0 p1 c0 {1,S} {3,S}\n3  C u0 p0 c0 {2,S} {4,S} {5,S} {6,S}\n4  F u0 p3 c0 {3,S}\n5  F u0 p3 c0 {3,S}\n6  C u0 p0 c0 {3,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 F u0 p3 c0 {9,S}\n']
species_structure_dict_leftover['rPF4acylF']= ['O=C(F)C(F)(F)C(F)(F)[C](F)F', 'multiplicity 2\n1  O u0 p2 c0 {2,D}\n2  C u0 p0 c0 {1,D} {3,S} {4,S}\n3  F u0 p3 c0 {2,S}\n4  C u0 p0 c0 {2,S} {5,S} {6,S} {7,S}\n5  F u0 p3 c0 {4,S}\n6  F u0 p3 c0 {4,S}\n7  C u0 p0 c0 {4,S} {8,S} {9,S} {10,S}\n8  F u0 p3 c0 {7,S}\n9  F u0 p3 c0 {7,S}\n10 C u1 p0 c0 {7,S} {11,S} {12,S}\n11 F u0 p3 c0 {10,S}\n12 F u0 p3 c0 {10,S}\n']
species_structure_dict_leftover['rPF4acid']= ['O=C(O)C(F)(F)C(F)(F)[C](F)F', 'multiplicity 2\n1  O u0 p2 c0 {2,D}\n2  C u0 p0 c0 {1,D} {3,S} {4,S}\n3  O u0 p2 c0 {2,S} {13,S}\n4  C u0 p0 c0 {2,S} {5,S} {6,S} {7,S}\n5  F u0 p3 c0 {4,S}\n6  F u0 p3 c0 {4,S}\n7  C u0 p0 c0 {4,S} {8,S} {9,S} {10,S}\n8  F u0 p3 c0 {7,S}\n9  F u0 p3 c0 {7,S}\n10 C u1 p0 c0 {7,S} {11,S} {12,S}\n11 F u0 p3 c0 {10,S}\n12 F u0 p3 c0 {10,S}\n13 H u0 p0 c0 {3,S}\n']
species_structure_dict_leftover['rPF5-1ene']= ['FC(F)=C(F)C(F)(F)C(F)(F)[C](F)F', 'multiplicity 2\n1  F u0 p3 c0 {2,S}\n2  C u0 p0 c0 {1,S} {3,S} {4,D}\n3  F u0 p3 c0 {2,S}\n4  C u0 p0 c0 {2,D} {5,S} {6,S}\n5  F u0 p3 c0 {4,S}\n6  C u0 p0 c0 {4,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 C u1 p0 c0 {9,S} {13,S} {14,S}\n13 F u0 p3 c0 {12,S}\n14 F u0 p3 c0 {12,S}\n']
species_structure_dict_leftover['PF5-1ene']= ['FC(F)=C(F)C(F)(F)C(F)(F)C(F)(F)F','1  F u0 p3 c0 {2,S}\n2  C u0 p0 c0 {1,S} {3,S} {4,D}\n3  F u0 p3 c0 {2,S}\n4  C u0 p0 c0 {2,D} {5,S} {6,S}\n5  F u0 p3 c0 {4,S}\n6  C u0 p0 c0 {4,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 C u0 p0 c0 {9,S} {13,S} {14,S} {15,S}\n13 F u0 p3 c0 {12,S}\n14 F u0 p3 c0 {12,S}\n15 F u0 p3 c0 {12,S}\n']
species_structure_dict_leftover['PF5s']=['F[C]C(F)(F)C(F)(F)C(F)(F)C(F)(F)F','multiplicity 1\n1  F u0 p3 c0 {2,S}\n2  C u0 p1 c0 {1,S} {3,S}\n3  C u0 p0 c0 {2,S} {4,S} {5,S} {6,S}\n4  F u0 p3 c0 {3,S}\n5  F u0 p3 c0 {3,S}\n6  C u0 p0 c0 {3,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 C u0 p0 c0 {9,S} {13,S} {14,S} {15,S}\n13 F u0 p3 c0 {12,S}\n14 F u0 p3 c0 {12,S}\n15 F u0 p3 c0 {12,S}\n']
species_structure_dict_leftover['rPF5acylF']=['O=C(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['rPF5acid']=['O=C(O)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['rPF6-1ene']=['FC(F)=C(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['PF6-1ene']=['FC(F)=C(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF6s']=['F[C]C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F','multiplicity 1\n1  F u0 p3 c0 {2,S}\n2  C u0 p1 c0 {1,S} {3,S}\n3  C u0 p0 c0 {2,S} {4,S} {5,S} {6,S}\n4  F u0 p3 c0 {3,S}\n5  F u0 p3 c0 {3,S}\n6  C u0 p0 c0 {3,S} {7,S} {8,S} {9,S}\n7  F u0 p3 c0 {6,S}\n8  F u0 p3 c0 {6,S}\n9  C u0 p0 c0 {6,S} {10,S} {11,S} {12,S}\n10 F u0 p3 c0 {9,S}\n11 F u0 p3 c0 {9,S}\n12 C u0 p0 c0 {9,S} {13,S} {14,S} {15,S}\n13 F u0 p3 c0 {12,S}\n14 F u0 p3 c0 {12,S}\n15 C u0 p0 c0 {12,S} {16,S} {17,S} {18,S}\n16 F u0 p3 c0 {15,S}\n17 F u0 p3 c0 {15,S}\n18 F u0 p3 c0 {15,S}\n']
species_structure_dict_leftover['rPF6acylF']=['O=C(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['rPF6acid']=['O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['PF7-1ene1r']=['F[C]=C(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF7-1ene2r']=['FC(F)=[C]C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF7-1ene3r']=['F[C](C(F)=C(F)F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF7-1ene4r']=['F[C](C(F)(F)C(F)=C(F)F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF7-1ene5r']=['F[C](C(F)(F)C(F)(F)F)C(F)(F)C(F)(F)C(F)=C(F)F']
species_structure_dict_leftover['PF7-1ene6r']=['F[C](C(F)(F)F)C(F)(F)C(F)(F)C(F)(F)C(F)=C(F)F']
species_structure_dict_leftover['rPF7-1ene']=['F[C](F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)=C(F)F']
species_structure_dict_leftover['PF7-1ene']=['FC(F)=C(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['PF7s']=['[C]C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']
species_structure_dict_leftover['rPF7acylF']=['O=C(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['rPF7acid']=['O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']
species_structure_dict_leftover['rPF8acid']=['O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)[C](F)F']


In [27]:
print(final_string)



entry(
    index = 0,
    label = "H",
    molecule = 
"""
multiplicity 2
1 H u1 p0 c0

""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs=[3.1682671, -0.00327931884, 6.64306396e-06, -6.12806624e-09, 2.11265971e-12, 29122.2592, 2.05193346], Tmin=(200.0,'K'), Tmax=(6000.0,'K')),
            NASAPolynomial(coeffs=[2.5, 0.0, 0.0, 0.0, 0.0, 25473.66, -0.44668285], Tmin=(6000.0,'K'), Tmax=(6000.0,'K')),
        ],
        Tmin = (200.0,'K'),
        Tmax = (6000.0,'K'),
        Cp0 = (26.342438367274774,'J/(mol*K)'),
    ),
    reference = 'GBR3M 
 **********************  Atomic species ***************************************',
    referenceType = "Theory",
)


entry(
    index = 1,
    label = "O",
    molecule = 
"""
multiplicity 3
1 O u2 p2 c0

""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs=[3.1682671, -0.00327931884, 6.64306396e-06, -6.12806624e-09, 2.11265971e-12, 29122.2592, 2.05193346], Tmin=(200.0,'K'), Tmax=(1000.0